In [1]:
import numpy as np
from sklearn.cluster import KMeans
from sklearn.metrics import mean_squared_error
import matplotlib.pyplot as plt

# Generate random 2D data
np.random.seed(42)
X = np.random.rand(20, 2)
Y = np.sin(X[:, 0] * 10) + np.cos(X[:, 1] * 5)
Y = Y.reshape(-1, 1)

# --- Kernel Functions ---
def rbf_kernel(x1, x2, sigma=1.0):
    return np.exp(-np.linalg.norm(x1 - x2) ** 2 / (2 * (sigma ** 2)))

def poly_kernel(x1, x2, degree=3, coef0=1):
    return (np.dot(x1, x2.T) + coef0) ** degree

# --- RBF Network Setup ---
kmeans = KMeans(n_clusters=6, n_init=10).fit(X)
centers = kmeans.cluster_centers_

from scipy.spatial.distance import cdist
d_max = np.max(cdist(centers, centers))
sigma = d_max / np.sqrt(2 * len(centers))

def build_kernel_matrix(X, centers, kernel_func, **kwargs):
    K = np.zeros((X.shape[0], len(centers)))
    for i in range(X.shape[0]):
        for j in range(len(centers)):
            K[i, j] = kernel_func(X[i], centers[j], **kwargs)
    return K

# RBF Model
R_rbf = build_kernel_matrix(X, centers, rbf_kernel, sigma=sigma)
W_rbf = np.dot(np.linalg.pinv(R_rbf), Y)
y_pred_rbf = np.dot(R_rbf, W_rbf)
mse_rbf = mean_squared_error(Y, y_pred_rbf)

# Polynomial Kernel Model
R_poly = build_kernel_matrix(X, centers, poly_kernel, degree=3, coef0=1)
W_poly = np.dot(np.linalg.pinv(R_poly), Y)
y_pred_poly = np.dot(R_poly, W_poly)
mse_poly = mean_squared_error(Y, y_pred_poly)

print(f"RBF Kernel MSE: {mse_rbf:.4f}")
print(f"Polynomial Kernel MSE: {mse_poly:.4f}")


RBF Kernel MSE: 0.2510
Polynomial Kernel MSE: 0.1928


| Kernel Function        | MSE    | Observation                                                                                    |
| ---------------------- | ------ | ---------------------------------------------------------------------------------------------- |
| **RBF (Gaussian)**     | 0.2510 | Performs well, capturing localized non-linear patterns. Slight underfitting seen.              |
| **Polynomial (deg=3)** | 0.1928 | Performs better here, likely due to mild global trends in random data. Shows more flexibility. |

 Conclusion:
- Although RBF is known for precise modeling of local relationships, in this case, Polynomial kernel surprisingly achieved a lower MSE, indicating it fit the generated random data slightly better.


- This highlights the importance of trying multiple kernels and validating with error metrics like MSE.